In [9]:
import os
import keyring
from functools import partial
from typing import Annotated, Sequence, TypedDict, Literal
import yfinance as yf 
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, BaseMessage
from pydantic import BaseModel
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent
import functools
import operator
from display_graph import display_graph

# API KEY
OPENAI_API_KEY = keyring.get_password('openai', 'key_for_mac')
TAVILY_API_KEY = keyring.get_password('tavily', 'key_for_mac')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

# Tavily tool
tavily_tool = TavilySearchResults(max_results=5)

# Set up LangSmith observability
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY'] = keyring.get_password('langsmith', 'learning_agent')
os.environ['LANGCHAIN_PROJECT'] = "pr-stupendous-hood-8"

In [14]:
# LLM definition
llm = ChatOpenAI(model='gpt-4o-mini')

# Route responses structure for supervisor's decision
class RouteResponseFin(BaseModel):
    next: Literal["Market_Data_Agent", "Analysis_Agent", "News_Agent", "FINISH"]

# Define agent members
members_fin = ["Market_Data_Agent", "Analysis_Agent", "News_Agent"]

# Supervisor prompt setup
system_prompt_fin = (
    "You are a Financial Service Supervisor managing the following agents: "
    f"{', '.join(members_fin)}. Select the next agent to handle the current query."
)

prompt_fin = ChatPromptTemplate.from_messages([
    ("system", system_prompt_fin),
    MessagesPlaceholder(variable_name="messages"),
    ("system", "Choose the next agent from: {options}."),
]).partial(options=str(members_fin))

# Supervisor Agent
def supervisor_agent_fin(state):
    supevisor_chain_fin = prompt_fin | llm.with_structured_output(RouteResponseFin)
    return supevisor_chain_fin.invoke(state)

In [15]:
# Define Tools and Agents Prompt

# 1. Market Data Tool and Agent Prompt
def fetch_stock_price(query):
    """Fetch the current stock price of a given stock symbol."""
    stock_symbol = query.split()[-1]
    stock = yf.Ticker(stock_symbol)
    try:
        current_price = stock.info.get("currentPrice")
        return f"The current stock price of {stock_symbol}: ${current_price}"
    except Exception as e:
        return f"Error retrieving stock data for {stock_symbol}: {str(e)}"
    
def agent_node(state, agent, name):
    result = agent.invoke(state)
    print(f"{name} Output: {result['messages'][-1].content}")
    return {
        "messages": [HumanMessage(content=result["messages"][-1].content, name=name)]
    }

marekt_data_prompt = (
    "You are the Market Data Agent. Your role is to retrieve latest stock prices or "
    "market information based on user queries. Ensure your repsonse includes the current price "
    "and any relevant market details if available."
)

market_data_agent = create_react_agent(llm, tools=[fetch_stock_price], state_modifier=marekt_data_prompt)
market_data_node = functools.partial(agent_node, agent=market_data_agent, name="Market_Data_Agent")

In [16]:
# 2. Financial Analysis Tool and Agent Prompt
def perform_financial_analysis(query):
    """Perform financial analysis based on user query."""
    if "ROI" in query:
        initial_investment = 1000
        final_value = 1200
        roi = ((final_value - initial_investment) / initial_investment) * 100
        return f"For an initial investment of ${initial_investment} yielding ${final_value}, the ROI is {roi}%."
    return "No relevant financial analysis found."

analysis_prompt = (
    "You are the Finanacial Analysis Agent. Analyze the financial data provided in the query. "
    "Perform calculations like ROI, growth rate, or other financial metrics as required. "
    "Provide a clear an concise response. "
    "Only use the following tools: "
    "perfom_financial_analysis"
)

analysis_agent = create_react_agent(llm, tools=[perform_financial_analysis], state_modifier=analysis_prompt)
analysis_node = functools.partial(agent_node, agent=analysis_agent, name="Analysis_Agent")

In [17]:
# 3. Finanacial News Tool and Agent Prompt
financial_news_tool = TavilySearchResults(max_results=5)
news_prompt = (
    "You are the Financial News Agent. Retrieve the latest news articles relevant to the user's query. "
    "Use search tools to gather up-to-date information and summarize key points. "
    "Do not quote sources, just give a summary."
)
financial_news_agent = create_react_agent(llm, tools=[financial_news_tool], state_modifier=news_prompt)
financial_news_node = functools.partial(agent_node, agent=financial_news_agent, name="News_Agent")


In [19]:
# Define workflow state
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str

# Define the workflow with the supervisor and agent nodes
workflow_fin = StateGraph(AgentState)
workflow_fin.add_node("Market_Data_Agent", market_data_node)
workflow_fin.add_node("Analysis_Agent", analysis_node)
workflow_fin.add_node("News_Agent", financial_news_node)
workflow_fin.add_node("Supervisor", supervisor_agent_fin)

# Define edges for agents to return to the supervisor
for member in members_fin:
    workflow_fin.add_edge(member, "Supervisor")

# Conditional map for routing based on supervisor's decision
conditional_map_fin = {
    "Market_Data_Agent": "Market_Data_Agent",
    "Analysis_Agent": "Analysis_Agent",
    "News_Agent": "News_Agent",
    "FINISH": END
}
workflow_fin.add_conditional_edges("Supervisor", lambda x: x["next"], conditional_map_fin)
workflow_fin.add_edge(START, "Supervisor")

# Compile the graph
graph_fin = workflow_fin.compile()

# Visualize the graph
display_graph(graph_fin)

/Users/woojin/Documents/github/learning/llm/langgraph/langgraph_blueprint/ch13/graphs/graph_73626.png


In [20]:
# Testing the workflow with an example input
inputs_fin = {"messages": [HumanMessage(content="What is the stock price of AAPL?")]}
for output in graph_fin.stream(inputs_fin):
    if "__end__" not in output:
        print(output)

{'Supervisor': {'next': 'Market_Data_Agent'}}
Market_Data_Agent Output: The current stock price of Apple Inc. (AAPL) is $229.98.
{'Market_Data_Agent': {'messages': [HumanMessage(content='The current stock price of Apple Inc. (AAPL) is $229.98.', additional_kwargs={}, response_metadata={}, name='Market_Data_Agent')]}}
{'Supervisor': {'next': 'FINISH'}}
